# Dependences

In [1]:
include("../../src/struct_data.jl")
include("../../src/neighbor.jl")
include("../../src/forces/forces.jl")

cu_force (generic function with 1 method)

# Model

In [2]:
@time Model = ModelSet(
    TimeModel(
        tₛᵢₘ  = 10000.0,
        dt    = 0.5,
        nₖₙₙ  = 50,
        nₛₐᵥₑ = 100
    ),
    InputModel(
        outer_ratio = 0.8,
        path_input = "../../data/init/Sphere"
    )
)
dump(Model)

  0.000002 seconds (3 allocations: 112 bytes)
ModelSet
  Time: TimeModel
    tₛᵢₘ: Float64 10000.0
    dt: Float64 0.5
    nₖₙₙ: Int64 50
    nₛₐᵥₑ: Int64 100
  Input: InputModel
    outer_ratio: Float64 0.8
    path_input: String "../../data/init/Sphere"


# Aggregate

In [3]:
FusionAGG = Nothing
@time FusionAGG = FusionAggregate(
    [
        AggType(
            "HEK_1", 
            InteractionPar(
                Cubic(0.01,2.0,3.0), 
                ContractilePar(0.001)
            ),
            Float64.(readdlm("../../data/init/Sphere/15.0.xyz")[3:end,2:end]) |> cu
        ),
        AggType(
            "HEK_2", 
            InteractionPar(
                Cubic(0.01,2.0,3.0), 
                ContractilePar(0.001)
            ),
            Float64.(readdlm("../../data/init/Sphere/15.0.xyz")[3:end,2:end]) |> cu
        )
    ], 
    Model
)
show_aggregates(FusionAGG)

  7.608521 seconds (13.11 M allocations: 1.133 GiB, 3.67% gc time, 4.80% compilation time)
========================= Type =======================


2-element Vector{AggType}:
 AggType("HEK_1", InteractionPar(Cubic{Float64}(0.01, 2.0, 3.0), ContractilePar(0.001)), 15.27f0, Float32[-1.5 -4.62 -13.88; 0.5 -4.62 -13.88; … ; 0.5 4.62 13.88; 2.5 4.62 13.88], CuArray{Float32, 2, CUDA.Mem.DeviceBuffer})
 AggType("HEK_2", InteractionPar(Cubic{Float64}(0.01, 2.0, 3.0), ContractilePar(0.001)), 15.27f0, Float32[-1.5 -4.62 -13.88; 0.5 -4.62 -13.88; … ; 0.5 4.62 13.88; 2.5 4.62 13.88], CuArray{Float32, 2, CUDA.Mem.DeviceBuffer})

======================   Index =======================
Index of List of Aggregates


1×5008 CuArray{Int64, 2, CUDA.Mem.DeviceBuffer}:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  2  2  2  2  2  2  2  2  2  2  2  2

Index of Number of Aggregates


...

Contractile


ContractilePar(Float32[0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001  …  0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001])

Radius


2-element CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}:
 15.27
 15.27

------------------ Neighbors Size -------------------
idx      = (5008, 5008)
idx_red  = (21, 5008)
idx_sum  = (1, 5008)
idx_cont = (50, 5008)
------------------- Forces Size ---------------------
dX       = (5008, 3)
F        = (5008, 3)


# kNN

In [4]:
println("------------------------ SIZE ------------------------")
println("r_max         = $(FusionAGG.Type[1].Interaction.Force.rₘₐₓ)")
println("Size X        = $(size(FusionAGG.Position))")
println("Size idx      = $(size(FusionAGG.Simulation.Neighbor.idx))")
println("Size idx_real  = $(size(FusionAGG.Simulation.Neighbor.idx))")
println("Size idx_sum   = $(size(FusionAGG.Simulation.Neighbor.idx_sum))")
println("Size idx_red   = $(size(FusionAGG.Simulation.Neighbor.idx_red))")
@time nearest_neighbors(FusionAGG)
println("------------------------ IDX -------------------------")
println("idx")
display(FusionAGG.Simulation.Neighbor.idx)
println("idx_red")
display(FusionAGG.Simulation.Neighbor.idx_red)
println("idx_sum")
display(FusionAGG.Simulation.Neighbor.idx_sum)
println("idx_cont")
display(FusionAGG.Simulation.Neighbor.idx_cont)

------------------------ SIZE ------------------------
r_max         = 3.0
Size X        = (5008, 3)
Size idx      = (5008, 5008)
Size idx_real  = (5008, 5008)
Size idx_sum   = (1, 5008)
Size idx_red   = (21, 5008)
Threads = (32, 32) | Blocks  = (157, 157)
Threads = 256 | Blocks  = 20
Threads = (32, 32) | Blocks  = (157, 157)
 19.560723 seconds (31.21 M allocations: 1.773 GiB, 3.17% gc time, 17.33% compilation time)
------------------------ IDX -------------------------
idx


5008×5008 CuArray{Int64, 2, CUDA.Mem.DeviceBuffer}:
 1  1  0   0   1   1   0   0   0   0  …     0     0     0     0     0     0
 2  2  2   0   0   2   2   0   0   0        0     0     0     0     0     0
 0  3  3   0   0   0   3   3   0   0        0     0     0     0     0     0
 0  0  0   4   4   0   0   0   4   4        0     0     0     0     0     0
 5  0  0   5   5   5   0   0   0   5        0     0     0     0     0     0
 6  6  0   0   6   6   6   0   0   0  …     0     0     0     0     0     0
 0  7  7   0   0   7   7   7   0   0        0     0     0     0     0     0
 0  0  8   0   0   0   8   8   0   0        0     0     0     0     0     0
 0  0  0   9   0   0   0   0   9   9        0     0     0     0     0     0
 0  0  0  10  10   0   0   0  10  10        0     0     0     0     0     0
 0  0  0   0  11  11   0   0   0  11  …     0     0     0     0     0     0
 0  0  0   0   0  12  12   0   0   0        0     0     0     0     0     0
 0  0  0   0   0   0  13  13   0   0

idx_red


21×5008 CuArray{Int64, 2, CUDA.Mem.DeviceBuffer}:
  1   1   2   4   1   1   2   3   4  …  4949  4950  4951  4957  4958  4959
  2   2   3   5   4   2   3   7   9     4957  4958  4959  4964  4965  4966
  5   3   7   9   5   5   6   8  10     4958  4959  4960  4965  4966  4967
  6   6   8  10   6   6   7  13  15     4964  4965  4966  4971  4972  4973
 35   7  37  40  10   7   8  14  48     4965  4966  4967  4972  4973  4974
 36  36  38  41  11  11  12  44  49  …  4966  4967  4968  4973  4974  4975
 37  37  39  42  41  12  13  45  50     4997  4998  4999  5002  5003  5004
 42  38  44  49  42  42  43  46  57     4998  4999  5000  5003  5004  5005
 43  43  45  50  43  43  44  53  58     5002  5003  5004  5006  5006  5007
 51  44  53  58  50  44  45  54  66     5003  5004  5005  5007  5007  5008
  0  52   0   0  51  51  52  62   0  …  5004  5005  5008     0  5008     0
  0   0   0   0  59  52  53   0   0     5006  5007     0     0     0     0
  0   0   0   0   0  60  61   0   0     5007  5008

idx_sum


1×5008 CuArray{Int64, 2, CUDA.Mem.DeviceBuffer}:
 10  11  10  10  12  13  13  11  10  …  11  10  12  13  13  11  10  11  10

idx_cont


50×5008 CuArray{Int64, 2, CUDA.Mem.DeviceBuffer}:
 36  37   8  41  51   7   8  13  57  …  4964  5005  5000  5003  4965  5007
 42   6   3  10   6  51   7  45  48     5006  5008  4999  4964  4974  5008
 36  38  38   9  50   1  13  14  48     4949  4999  5004  4957  4974  5007
 37  52  44  50  10  43   8  14  57     4965  4999  4967  5007  4958  4974
 37   1   8   5   4  11   6  44  50     4964  4998  5004  5007  5008  5005
  5   2  53  42  43  43  45  62   9  …  4949  4999  4966  5006  5007  4967
  1   3   8   4  51   2   7  46   4     4998  4998  4967  4973  5007  5007
  5   1  44   5  59  11   7  53  50     4965  4966  5000  5006  4965  4959
 35   3  38  49  10   7  45   7  57     5004  4967  4966  5002  4973  5008
  1   3   7  40  41  44  13  14   9     4998  4958  4968  4972  5004  4959
  2  38   8   4  51   1   8  45  58  …  4997  4999  5004  4972  4965  5005
 37  38  44   9  50   7  52   7  15     4958  5003  4999  5002  5004  4959
  5   3   3  42   5  43  44  46   4     5007  5008

# Force

In [8]:
println("Force Parameter")
display(FusionAGG.Simulation.Parameter.Force)
println("Contractile Parameter")
display(FusionAGG.Simulation.Parameter.Contractile.fₚ')
println("Running CUDA kernel")
@time cu_force(FusionAGG,1.0)
println("Force")
display(FusionAGG.Simulation.Force.F)

Force Parameter


Cubic{CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}}(Float32[0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01  …  0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01], Float32[2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0  …  2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0], Float32[3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0  …  3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0])

Contractile Parameter


1×5008 adjoint(::CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}) with eltype Float32:
 0.001  0.001  0.001  0.001  0.001  …  0.001  0.001  0.001  0.001  0.001

Running CUDA kernel
  0.000089 seconds (48 allocations: 3.250 KiB)
Force


5008×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
  0.0           0.000561564  -0.000452452
  0.0           0.000561564  -0.000452452
 -0.000500444  -0.000880691   0.000364654
  0.0           0.000710227  -0.000452453
 -0.000509339  -0.000141072  -0.00045103
 -0.001         0.000164602   0.000364654
 -0.001         0.000164602   0.000364654
  0.000507203  -0.000717842   0.000364654
  0.000526075  -0.000370077  -0.000452033
  0.000706996   0.000251302  -0.000211375
 -0.000706996   0.000251302  -0.000211375
  0.0          -0.000158756   0.000364828
 -0.001        -0.000158756   0.000364828
  ⋮                          
  0.0           0.000158756  -0.000364828
  0.000501141   0.000444407   0.000452033
 -0.001         0.000158756  -0.000364828
 -3.38513f-5    0.00091574    0.000213387
  0.000498284   0.000733273  -0.000364654
  0.000991081  -0.000149172  -0.000364654
  0.000708019  -0.000571713   0.000212382
 -0.001        -0.000164602  -0.000364654
 -0.000489364   0.000717842  -0.000364